# Boto for Bozos: Lesson 2
### Creating Basic VPC Resources Using Boto3

___

Welcome to the Boto for Bozos lesson 2! In this lesson, we're going to start deploying VPC resources! We'll configure a VPC, tag it, and add subnets using functions that will allow us to deploy multiple subnets using only one call. This may not be an ideal setup for everyone, but it should work well for most cases!

If you have followed along with the first guides, you should already have Jupyter Notebook installed. If not, check out the first guide here: https://www.bravethecloud.com/guides/getting-started/ if you wish to follow along in your own Jupyter Notebook!

*Note: Press "ctrl+enter" to run each cell and move to the next.

### First, let's upgrade pip
(skip if you have already performed this)

In [ ]:
!pip install --upgrade pip

### Now, let's install boto3, cython, and pprint using pip
I purposefully left the boto3 install off of the Jupyter setup so we can perform it explicitly. I also added "cython", which will cause the install to throw an error if not included. 'pprint' is in the standard library, but I've added it just in case.
(skip if you have already performed this)

In [ ]:
!pip install boto3 cython pprint

### Let's import boto3:

In [468]:
import boto3
import pandas as pd
from pprint import pprint

### Let's create a non-default VPC:
Using the default VPC is great for PoCing a project or testing things out quickly, but it is never recommended for a production workload, so let's create a new VPC! 

### First, we need to set our region and our CIDR range:
I'm going to use us-east-2 for this, but if you wish to change it, you can. Be aware, some regions have more or fewer availability zones in them, so you may have to modify the scripts accordingly. I choose "10.152.0.0/16" instead of something simple like "10.0.0.0/16" because I want to ensure I have a very low probability of creating a VPC that overlaps with a home network, office network, or any other network that may need to peer or create a VPN connection. 

In [347]:
region = 'us-east-2'
cidr = '10.152.0.0/16'

### Now, to setup the boto3 session:

In [348]:
ec2 = boto3.client('ec2', region_name=region)

### Then, we create the VPC! 
As you can see, I've commented out a few things. This is just to show you some of the other possibilities. I don't show this in all commands, but creating a VPC is obviously extremely important! 

In [349]:
vpc = ec2.create_vpc(
    CidrBlock=cidr
    #AmazonProvidedIpv6CidrBlock=True|False, ### This will enable IPv6 on your VPC. You cannot choose that CIDR. 
    #DryRun=True|False, ### This will just test for proper permissions to manage the VPC. 
    #InstanceTenancy='default'|'dedicated'|'host' ### This determines the type of EC2 tenancy. Check out the AWS docs for more!
)
#pprint(vpc) # Feel free to print the vpc object, since we performed this in the last lesson, I'll leave it commented out. 

### Let's get our VPC ID:
In this example, we use the VPC CIDR to obtain the VPC ID. Obviously, this wouldn't work if you had multiple VPCs with the same CIDR, but you don't do that, do you? You prefer best practices! 

In [350]:
vpc = ec2.describe_vpcs(
    Filters=[
        {
            'Name': 'cidr',
            'Values': [
                cidr
            ]
        }
    ]
)
vpcId = (vpc['Vpcs'][0]['VpcId'])
print(vpcId)

vpc-02499d051374f44bf


### Getting the VPC by CIDR is one way, but tags are the best way, so let's create one:

In [351]:
ec2.create_tags(
    Resources = [
    vpcId
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'prodVPC'
        },
    ]

)

{'ResponseMetadata': {'RequestId': '06354cff-dead-4dd6-a1fd-44eba8d862bf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'content-length': '221',
   'date': 'Fri, 16 Nov 2018 06:10:03 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

### Now we can query the VPC based on its tag:

In [352]:
vpc = ec2.describe_vpcs(
    Filters=[
        {
            'Name': 'tag:Name',
            'Values': [
                'prodVPC'
            ]
        }
    ]
)
vpcId = (vpc['Vpcs'][0]['VpcId'])
print(vpcId)

vpc-02499d051374f44bf


### Ok, so now the VPC is available, let's add some subnets!
First, I'm going to check for the available AZs in my region. As you can see, US-East-2 only has three: 

In [353]:
avaz = ec2.describe_availability_zones()
for zones in avaz['AvailabilityZones']:
    print(zones['ZoneName'])

us-east-2a
us-east-2b
us-east-2c


### Now to populate my "az" list and my "snNames" list:
These two lists will be used to deploy the subnets. As you can see, I'm going to use all of the AZs available in my region and configure four types of subnets: "Public", "Private", "DB", and "Dev". We will eventually configure appropriate route tables and security rules to correspond with these monikers. 

In [354]:
az = ['a', 'b', 'c']
snNames = ['public', 'private', 'db', 'dev']

### This function will be used for the subnet creation:
The function will take the VPC CIDR and modify it to suit the subnets based on the number of subnets we create, the base network, and the prefix we will use for our subnets. 

In [505]:
def subnetCidr(n, cidr, prefix):
    newCidr = cidr.split('.')
    newCidr[2] = str(n)
    s = '.'
    cidr = s.join(newCidr).split('/')[0]
    return('{}/{}'.format(cidr,prefix))

### This is an example of the output:
As you can see, we pass in "5", the current "cidr" value, and 24 for the "prefix". 

In [508]:
print(cidr)
subnetCidr(5, cidr, 24)

10.152.0.0/16


'10.152.5.0/24'

___

### Now to create our subnets!
In this cell, we are going to iterate through the available subnet names, this will determine how many subnets are created. Since we have 4 subnet names ("public", 'private", "db", and "dev"), this loop will run 4 times. 

What I've done in the first for loop, is I have created empty lists based on the subnet names, these lists will be populated with the subnets we create. Each list will contain a subnet from each availability zone. So there will be a subnet from US-East-2a, b, and c in each of the four lists named after our subnets. 

After that, I loop over the subnet names again, but this time, each subnet name will have 3 subnets created, one in each availability zone. 

I then use our subnetCidr function to pass in the cidr, prefix, and "n", the current iteration, in order to create and number the subnets. I also pass in the vpcId we obtained before. 


In [419]:
for name in snNames:
    vars()[name] = []
n = 1  
for subnet in snNames:
    i = -len(az)
    while i < 0:
        network = ec2.create_subnet(
            CidrBlock = subnetCidr(n, cidr, 24),
            AvailabilityZone= '{}{}'.format(region, str(az[i])),
            VpcId= vpcId
        )
        vars()[subnet].append(network['Subnet']['SubnetId'])
        n += 1
        i += 1

Note: Currently, this increments the 3rd octet of the CIDR by 1. This is good for a /24, but not for other prefixes. I hope to create a function that adapts for other subnets, but for now, this is what we've got. Feel free to reach out if you have a solution ready to go, I'll gladly give props! 
___

### Once the subnets have been created, we can print out the lists:
As you can see, each list has an equal number of subnets, and if you query them, you will see they are evenly distributed among the availability zones. 

In [420]:
print(public)
print(private)
print(db)
print(dev)

['subnet-0bb1388bde0f1685b', 'subnet-09c0eae17eb87042b', 'subnet-07793887c819fac8e']
['subnet-07d1cd0a0841d93d1', 'subnet-0194b61c7645f2ea1', 'subnet-0f887cf89c6aee3b3']
['subnet-0c26212d38b13b24c', 'subnet-059b2ba7d59f70046', 'subnet-033b19e1d59bcae2f']
['subnet-01a769927992d05a8', 'subnet-0a4ef105b2a337e5e', 'subnet-06bac0aba826ebea5']


___

### Nice! We now have a VPC with plenty of subnets to work with!
So let's get these subnets properly tagged. Here, I have created another function that will tag the subnets based on the list that they're in. As you remember, each list includes subnets from all of our AZs.

In [490]:
def tagSubnetList(snList, tagName):
    i = 0
    for snId in snList:
        tags = ec2.create_tags(
                Resources=[
                    snId,
                ],
                Tags=[
                    {
                        'Key': 'Name',
                        'Value': "{} - {}".format(tagName, str(i))
                    }
                ]
            )
        i += 1

In [491]:
tagSubnetList(public, 'Public')

In [492]:
tagSubnetList(private, 'Private')

In [493]:
tagSubnetList(db, 'DB')

In [494]:
tagSubnetList(dev, 'Dev')

### Now, let's take our subnets, query them, and output them in a nice table! 

In [504]:
subnets = ec2.describe_subnets(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [
                vpcId
            ]
        }
    ]
)
number = len(subnets['Subnets'])
i = 0
item_list = []
while i < number:
    subnetName = subnets['Subnets'][i]['Tags'][0]['Value']
    subnetCidr = subnets['Subnets'][i]['CidrBlock']
    subnetAz = subnets['Subnets'][i]['AvailabilityZone']
    item = {'Name':subnetName, 'AZ':subnetAz, 'CIDR':subnetCidr}
    item_list.append(item)    
    i += 1
    
df = pd.DataFrame(data=item_list, columns=['Name','CIDR','AZ'])
df.sort_values(['AZ','Name'])
#df.reset_index(drop=True)

,Name,CIDR,AZ
1,DB - 0,10.152.7.0/24,us-east-2a
0,Dev - 0,10.152.10.0/24,us-east-2a
10,Private - 0,10.152.4.0/24,us-east-2a
5,Public - 0,10.152.1.0/24,us-east-2a
7,DB - 1,10.152.8.0/24,us-east-2b
6,Dev - 1,10.152.11.0/24,us-east-2b
3,Private - 1,10.152.5.0/24,us-east-2b
8,Public - 1,10.152.2.0/24,us-east-2b
11,DB - 2,10.152.9.0/24,us-east-2c
2,Dev - 2,10.152.12.0/24,us-east-2c


### Great! We have now configured a non-default VPC and plenty of subnets to work with to ensure we maintain high availability and scalability, so this lesson is complete! Keep an eye on my blog at https://www.bravethecloud.com for future installments! 

### Command Review

##### To start creating ec2 resources, you need to setup the boto3 client:
ec2 = boto3.client('ec2', region_name='us-east-2')

##### Commands used in this tutorial:
ec2.create_vpc() <br>
ec2.create_subnet() <br>
ec2.describe_availability_zones() <br>
ec2.describe_instances() <br>
ec2.describe_vpcs() <br>
ec2.create_tags()